In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb # 브라우져를 제어하기 위한 도구
from selenium.webdriver.common.keys import Keys # 키보드를 제어하는 도구
import time # 시간 딜레이를 위한 모듈
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from flask import Flask
from flask import request, redirect,jsonify
op = wb.ChromeOptions()
op.add_argument('headless')
op.add_argument('disable-gpu')

In [2]:
e_data = pd.read_csv('data/감정단어 - 시트1.csv')
e_data_c = e_data.copy()
word_data = e_data_c['관련된 단어']
word_data1 = word_data[0].replace(' ','').split(",")
word_data2  = word_data[1].strip().split(",")
word_data3 = word_data[2].replace(' ','').split(",") 
word_data4 = word_data[3].replace(' ','').split(",")  
word_data5 = word_data[4].replace(' ','').split(",")  
word_data6 = word_data[5].replace(' ','').split(",")  
word_data7 = word_data[6].strip().replace(' ','').split(",")   
e_data2 = pd.read_csv('la3.csv')
e_data2_c = e_data2.copy()
X = e_data2_c.loc[:,'가사']
y = e_data2_c.loc[:,'라벨']
x_train,x_test,y_train,y_test = train_test_split(X,y,
                                               random_state = 50)

In [3]:
global word_cv1
word_cv1 = CountVectorizer(max_df=100, min_df=3) #데이터 정규화를 위한 CountVectorizer모델 사용

word_cv1.fit(x_train) #단어 학습

x_train_transformed1 = word_cv1.transform(x_train) # 텍스트 정규화
x_test_transformed1 = word_cv1.transform(x_test) # 텍스트 정규화

global word_logi1
#텍스트 마이닝에 효과적인 LogisticRegression모델 사용
word_logi1 = LogisticRegression(C=0.001,max_iter=1000000,random_state=50) 

#정규화 데이터와 라벨 데이터로 학습
word_logi1.fit(x_train_transformed1,y_train)

In [4]:
word_logi1.score(x_test_transformed1,y_test)

0.7516778523489933

In [4]:
#플라스크에 사용할 로직 정의
global e1,e2,e3,e4,e5,e6,e7
def code(title,singer):
    data_list = []
    #감정 점수화 변수
    # 분노:e1 즐거움:e2 행복:e3 슬픔:e4 불안:e5 사랑:e6 우울:e7
    global e1,e2,e3,e4,e5,e6,e7
    e1,e2,e3,e4,e5,e6,e7 = 0,0,0,0,0,0,0
    
    #사용자가 입력한 노래 제목과 가수를 웹드라이버를 통해 노래 가사 실시간 크롤링
    search=f'{singer}&{title}'
    url1 = f'https://www.genie.co.kr/search/searchMain?query={search}'      
    driver = wb.Chrome('chromedriver.exe',chrome_options=op)  
    driver.get(url1)
    time.sleep(1)
    
    driver.find_element_by_css_selector('.list>.link>.btn-basic').click()
    time.sleep(1)
    
    html = bs(driver.page_source, 'html.parser')
    
    a = html.select('#pLyrics p')[0]
    b = word_cv1.transform(a)
    time.sleep(1)
    driver.quit()
    
    #구현한 모델를 사용하여 감정 라벨 예측
    #분노:1 즐거움:2 행복:3 슬픔:4 불안:5 사랑:6 우울:7
    pre = str (word_logi1.predict(b)[0])
    
    #가지고 온 가사와 감정 단어 비교로 점수화
    str1 = a.text.replace('\t','').replace('\n','')
    for k in word_data1:
        if k in str1:
            e1+=1
    for k in word_data2:
        if k in str1:
            e2+=1
    for k in word_data3:
            if k in str1:
                e3+=1
    for k in word_data4:
            if k in str1:
                e4+=1
    for k in word_data5:
            if k in str1:
                e5+=1
    for k in word_data6:
            if k in str1:
                e6+=1
    for k in word_data7:
            if k in str1:
                e7+=1
    
    #점수화한 데이터로 색상 혼합하여 노래의 감정 색 추출
    header ={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36'         
    }
    
    driver1 = wb.Chrome('chromedriver.exe',chrome_options=op)
        #driver1 = wb.Chrome()
    url2 = 'https://trycolors.com/'
    driver1.get(url2)
    time.sleep(1)
    
        #분노(빨강):1,즐거움(노랑):2,행복(초록):3,슬픔(파랑):4,불안(하늘):5,사랑(하양):6,우울(검정):7
    for k in range(e2):#노랑
            driver1.find_element_by_css_selector('.palette_stack>.input_color>.mdl-button').click()
    for k in range(e1):#빨강
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color>.mdl-button').click()
    for k in range(e4):#파랑
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e5):#하늘
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e3):#초록
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e6):#분홍
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e7):#검정
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color+.input_color+.input_color.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    time.sleep(7)
    color = driver1.find_element_by_css_selector('.mix_wrapper>.color_hex').text
    time.sleep(1)
    driver.quit() 
    time.sleep(1)
    song_color = color[1:]
    url3 = f'http://localhost:8082/Project_Null/recieve_data?title={title}&singer={singer}&result={song_color}&emotion={pre}'
    data_list.append(song_color)
    data_list.append(title)    
    data_list.append(singer)    
    data_list.append(pre)

    return data_list

In [ ]:
#플라스크 사용
app = Flask(__name__) 

@app.route('/analysis', methods=['GET','POST'])
def index():  
    data_lst = []
    data_list = []
    cn = 0
    # 웹 상에서 반복횟수 변수 가지고 오기
    cn1 = int(request.args['dcnt'])
    
    #웹 상에서 사용자가 입력한 데이터 가지고 오기
    for k in range(0,cn1):
        title=request.args[f'tit{k+1}']
        singer=request.args[f'sin{k+1}']
        print(title)
        print(singer)
        data_list = code(title,singer)       
        data_lst.append(data_list[0])
        data_lst.append(data_list[1])
        data_lst.append(data_list[2])        
        data_lst.append(data_list[3])
    print(cn1)    
    print(data_lst) 
    
    c = 0;
    title=""
    singer=""
    em=""
    co=""
    
    #사용자에게 보내줄 데이터를 리스트 형태로 저장
    for k in range(0,cn1):
        title += f"{data_lst[1+c]}/"
        singer += f"{data_lst[2+c]}/"
        em += f"{data_lst[3+c]}/"
        co += f"{data_lst[0+c]}/"
        c+=4
    
    #쿼리스트링 방식으로 URL에 담아서 웹상으로 데이터 보내기
    url = f"http://localhost:8082/Project_Null/recieve_data?title={title}&singer={singer}&emotion={em}&code={co}"              
    
    #페이지 이동
    return redirect(url)
    
if __name__ == '__main__':
     app.run(host='127.0.1.1', port='5000')
        

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.1.1:5000/ (Press CTRL+C to quit)


헤픈 우연
헤이즈


<ipython-input-4-76730e59232a>:8: DeprecationWarning: use options instead of chrome_options
  driver = wb.Chrome('chromedriver.exe',chrome_options=op)
<ipython-input-4-76730e59232a>:44: DeprecationWarning: use options instead of chrome_options
  driver1 = wb.Chrome('chromedriver.exe',chrome_options=op)
127.0.0.1 - - [27/Jun/2021 10:31:38] "GET /analysis?tit1=헤픈+우연&dcnt=1&sin1=헤이즈 HTTP/1.1" 302 -


1
['805B56', '헤픈 우연', '헤이즈', '6']


In [47]:
app = Flask(__name__) 

@app.route('/analysis', methods=['GET','POST'])
def index():  
    data_lst = []
    data_list = []
    cn = int(request.args['dcnt'])
    print(type(cn))
if __name__ == '__main__':
     app.run(host='127.0.1.1', port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.1.1:5000/ (Press CTRL+C to quit)
[2021-06-21 10:48:36,646] ERROR in app: Exception on /analysis [GET]
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2097, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [21/Jun/2021 10:48:36] "GET /analysis?title=라일락&dcnt=2&singer=아이유 HTTP/1.1" 500 -


2


[2021-06-21 10:48:42,143] ERROR in app: Exception on /analysis [GET]
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2097, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [21/Jun/2021 10:48:42] "GET /analysis?title=라일락&dcnt=2&singer=아이유 HTTP/1.1" 500 -


<class 'int'>
